# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [26]:
import pandas as pd

df = pd.read_csv("sentiment140.csv", encoding='ISO-8859-1') 

df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']

# Create a smaller sample for development
sample_size = 20000 
df_sample = df.sample(n=sample_size, random_state=42)
df_sample

# target: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

,target,id,date,flag,user,text
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick"
...,...,...,...,...,...,...
1374482,4,2051447103,Fri Jun 05 22:02:36 PDT 2009,NO_QUERY,_Jaska,@girlwonder24 Thanks.
667014,0,2245469948,Fri Jun 19 16:10:39 PDT 2009,NO_QUERY,julianicolao,"trying to study for the biggest test, next wee..."
1451234,4,2063022808,Sun Jun 07 01:05:46 PDT 2009,NO_QUERY,ElaineToni,Just finished watching Your Song Presents: Boy...
1181412,4,1982082859,Sun May 31 10:29:36 PDT 2009,NO_QUERY,lindseyrd20,@janfran813 awww i can't wait to get one


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [27]:
import re

import nltk 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def clean_up(column):
    text = column.lower() # Convert to lowercase
    text = re.sub(r'\d+', ' ', text)   # Remove numbers
    text = re.sub(r'http\S+', ' ', text) # Remove URLs
    text = re.sub(r'[^\w\s]', ' ', text)  # Remove special characters (except spaces)
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces

    return text

def tokenize(column):
    tokens = word_tokenize(column)
    tokens = [word for word in tokens if word.isalnum()]
    
    return tokens

def stem_and_lemmatize(column):
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in column]
    
    return lemmatized

def remove_stopwords(column):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in column if word not in stop_words]
    
    return filtered_tokens

# Apply functions
df_sample['text_cleaned'] = df_sample['text'].apply(clean_up)
df_sample['text_token'] = df_sample['text_cleaned'].apply(tokenize)
df_sample['text_lemma'] = df_sample['text_token'].apply(stem_and_lemmatize)
df_sample['text_processed'] = df_sample['text_lemma'].apply(remove_stopwords)

df_sample = df_sample.reset_index()
df_sample

,index,target,id,date,flag,user,text,text_cleaned,text_token,text_lemma,text_processed
0,541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...,nkluvr eva my poor little dumpling in holmdel ...,"[nkluvr, eva, my, poor, little, dumpling, in, ...","[nkluvr, eva, my, poor, little, dumpling, in, ...","[nkluvr, eva, poor, little, dumpling, holmdel,..."
1,750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...,i m off too bed i gotta wake up hella early to...,"[i, m, off, too, bed, i, got, ta, wake, up, he...","[i, m, off, too, bed, i, got, ta, wake, up, he...","[bed, got, ta, wake, hella, early, tomorrow, m..."
2,766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...,i havent been able to listen to it yet my spea...,"[i, havent, been, able, to, listen, to, it, ye...","[i, havent, been, able, to, listen, to, it, ye...","[havent, able, listen, yet, speaker, busted]"
3,285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,now remembers why solving a relatively big equ...,"[now, remembers, why, solving, a, relatively, ...","[now, remembers, why, solving, a, relatively, ...","[remembers, solving, relatively, big, equation..."
4,705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick",ate too much feel sick,"[ate, too, much, feel, sick]","[ate, too, much, feel, sick]","[ate, much, feel, sick]"
...,...,...,...,...,...,...,...,...,...,...,...
19995,1374482,4,2051447103,Fri Jun 05 22:02:36 PDT 2009,NO_QUERY,_Jaska,@girlwonder24 Thanks.,girlwonder thanks,"[girlwonder, thanks]","[girlwonder, thanks]","[girlwonder, thanks]"
19996,667014,0,2245469948,Fri Jun 19 16:10:39 PDT 2009,NO_QUERY,julianicolao,"trying to study for the biggest test, next wee...",trying to study for the biggest test next week...,"[trying, to, study, for, the, biggest, test, n...","[trying, to, study, for, the, biggest, test, n...","[trying, study, biggest, test, next, week, wor..."
19997,1451234,4,2063022808,Sun Jun 07 01:05:46 PDT 2009,NO_QUERY,ElaineToni,Just finished watching Your Song Presents: Boy...,just finished watching your song presents boys...,"[just, finished, watching, your, song, present...","[just, finished, watching, your, song, present...","[finished, watching, song, present, boystown]"
19998,1181412,4,1982082859,Sun May 31 10:29:36 PDT 2009,NO_QUERY,lindseyrd20,@janfran813 awww i can't wait to get one,janfran awww i can t wait to get one,"[janfran, awww, i, can, t, wait, to, get, one]","[janfran, awww, i, can, t, wait, to, get, one]","[janfran, awww, wait, get, one]"


In [28]:
neg_tweets = df_sample[df_sample['target']==0]
neg_tweets.shape

(9942, 11)

In [29]:
neutral_tweets = df_sample[df_sample['target']==2]
neutral_tweets.shape

(0, 11)

In [30]:
pos_tweets = df_sample[df_sample['target']==4]
pos_tweets.shape

(10058, 11)

### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [33]:
from nltk.probability import FreqDist

all_words = [word for words in df_sample['text_processed'] for word in words]
freq_dist = FreqDist(all_words)
top_5000_words = freq_dist.most_common(5000)

top_5000_df = pd.DataFrame(top_5000_words)
top_5000_df

,0,1
0,wa,1400
1,day,1323
2,good,1175
3,get,1102
4,like,977
...,...,...
4995,regularly,3
4996,petewentz,3
4997,inspired,3
4998,nun,3


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [35]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Extract only the words from top_5000_words
top_words = [word for word, _ in top_5000_words]

# Create CountVectorizer with vocabulary restriction
vectorizer = CountVectorizer(vocabulary=top_words)

# Fit and transform to get the document-term matrix
dtm = vectorizer.fit_transform([' '.join(words) for words in df_sample['text_processed']])

# Convert sparse matrix to dense numpy array for easier manipulation
dtm_dense = dtm.toarray()

# Assuming your sentiment labels (0 for negative, 4 for positive) are in 'target' column
sentiment_labels = df_sample['target'].replace(4,1).to_numpy()

In [39]:
dtm_dense

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [41]:
dtm_dense.shape

(20000, 5000)

In [40]:
sentiment_labels

array([0, 0, 0, ..., 1, 1, 0], dtype=int64)

In [48]:
# Create a list to hold the features in the required format
feature_list = []

# Iterate through each document (row) in the matrix
for i in range(dtm_dense.shape[0]):

    # Create a dictionary for the document's word features
    document_dict = {word: (count > 0) for word, count in zip(top_words, dtm_dense[i, :])}
    
    # Add the document's feature dictionary and sentiment label as a tuple to the feature_list
    feature_list.append((document_dict, sentiment_labels[i]))

feature_list

[({'wa': True,
   'day': False,
   'good': False,
   'get': False,
   'like': False,
   'go': False,
   'quot': False,
   'love': False,
   'work': False,
   'got': False,
   'today': False,
   'going': False,
   'u': False,
   'time': False,
   'one': False,
   'lol': False,
   'know': False,
   'im': False,
   'back': False,
   'really': True,
   'want': False,
   'night': False,
   'amp': False,
   'well': False,
   'new': False,
   'see': False,
   'think': False,
   'still': False,
   'oh': False,
   'thanks': False,
   'na': False,
   'ha': False,
   'need': False,
   'home': False,
   'much': False,
   'miss': False,
   'feel': False,
   'last': False,
   'morning': False,
   'great': False,
   'make': False,
   'tomorrow': False,
   'twitter': False,
   'haha': False,
   'wish': False,
   'hope': True,
   'bad': False,
   'sad': False,
   'fun': False,
   'come': False,
   'sleep': False,
   'would': False,
   'nice': False,
   'sorry': False,
   'right': False,
   'week': Fals

#### Explanation

DTM: Document-Term Matrix

DTM stands for Document-Term Matrix. It's a fundamental data structure used in text analysis and natural language processing (NLP).  Here's how it works:

Documents: Each row in the matrix represents a single document (in your case, a tweet).
Terms: Each column represents a unique word (term) from your vocabulary (the top 5000 words in your case).
Values: The values in the matrix are the frequencies of each word in each document.
In simpler terms, a DTM shows you how many times each word appears in each document.

True/False in the feature_list:

The True/False values you see in the feature_list are a slightly different representation of the DTM.  Instead of raw word counts, they indicate the presence or absence of each word in a document:

True: The word is present in the document.
False: The word is absent from the document.
This binary representation is often used for text classification tasks because it focuses on whether a word appears at all, rather than how many times it appears.  It makes the features more suitable for algorithms like Naive Bayes that often work well with categorical data.

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [49]:
from nltk.classify import NaiveBayesClassifier
from sklearn.model_selection import train_test_split

# Splitting the data into training and test sets
train_set, test_set = train_test_split(feature_list, test_size=0.2, random_state=42)

# Training the Naive Bayes classifier
classifier = NaiveBayesClassifier.train(train_set)

# Displaying the most informative features
classifier.show_most_informative_features(10)

Most Informative Features:
Most Informative Features
                     ugh = True                0 : 1      =     23.3 : 1.0
                   upset = True                0 : 1      =     22.6 : 1.0
                   sadly = True                0 : 1      =     17.2 : 1.0
                   broke = True                0 : 1      =     15.6 : 1.0
                    poor = True                0 : 1      =     15.5 : 1.0
                congrats = True                1 : 0      =     14.8 : 1.0
                    died = True                0 : 1      =     14.0 : 1.0
                     sad = True                0 : 1      =     12.9 : 1.0
                   burnt = True                0 : 1      =     11.8 : 1.0
              depressing = True                0 : 1      =     11.1 : 1.0


#### Explanation
The output above provides insights into which words are most helpful in classifying your tweets as positive (1) or negative (0).

Here's a breakdown of what each part means:

E.g:       ugh = True               0 : 1      =     23.3 : 1.0
ugh = True: This means the presence of the word "ugh" in a tweet.
0 : 1: This indicates the sentiment categories being compared:
0: Negative sentiment
1: Positive sentiment
23.3 : 1.0: This is the likelihood ratio. It tells you how much more likely the word "ugh" is to appear in a negative tweet compared to a positive tweet.
Interpreting the Likelihood Ratio:

A likelihood ratio of 23.3 : 1.0 means that the word "ugh" is 23.3 times more likely to appear in a negative tweet than in a positive tweet.  This suggests that "ugh" is a strong indicator of negative sentiment.

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [50]:
accuracy = nltk.classify.accuracy(classifier, test_set)

accuracy

#Yey!
# An accuracy of 0.726 means that the classifier correctly predicted the sentiment of 72.6% of the tweets in the test set.  
# In other words, for every 100 tweets the model was given, it got about ~73 of them right.

0.726

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here